# Hive笔记
## 第二部分 数据操作和数据查询

　　ID：wlong
　　数据分析爱好者、目前小白一枚
　　邮箱：wlong826@163.com
　　和鲸社区：[【wlong】](https://www.heywhale.com/home/user/profile/5f0a61d72be199002c073935)
　　如果有不完善的地方，欢迎小伙伴评论区留言！
　　最近每周都会有所更新，欢迎大家关注点赞分享哦！
 
 ----
 
 　关于nowcoder上91道SQL题刷题总结：[【SQL实战 | nowcoder 91题总结 】](https://www.heywhale.com/mw/project/60f140e6486e5c00170b00f5) 

　如果对爬虫感兴趣的小伙伴，看这看这👉：[【零基础入门Python爬虫--豆瓣、58同城、安居客等】](https://www.heywhale.com/mw/project/60be1fa8acdcb3001707b133)

　关于Pandas数据处理的一些小总结，看这看这👉：[【Pandas数据处理120题知识点总结】](https://www.heywhale.com/mw/project/60c0b06919d601001898394c)

　经典数分案例泰坦尼克号的生存分析，看这看这👉：[【经典案例之泰坦尼克号乘客生存情况预测分析 】](https://www.heywhale.com/mw/project/60d18448056f570017c149dc)

　如果小伙伴对于Python实现数据结构感兴趣，可以看这👉：[【线性表】](https://www.heywhale.com/mw/project/60e7b58cbe5f09001761ac87)、[【栈和队列】](https://www.heywhale.com/mw/project/60ebe5de3f2448001705ccc2)

## 1. 数据操作

### 1.1 向管理表中装载数据

![Image Name](https://cdn.kesci.com/upload/image/qwh92ef9tr.png?imageView2/0/w/640/h/320)

　　如果分区目录不存在的话，这个命令会先创建分区目录，然后再将数据拷贝到该目录下；
　　
　　如果目标表是非分区表，那么语句中应该省略PARTITION子句；

### 1.2 通过查询语句向表中插入数据

　　INSERT语句允许用户通过**查询语句**向目标表中插入数据；
	
![Image Name](https://cdn.kesci.com/upload/image/qwhi9rupum.png?imageView2/0/w/480/h/320)

　　这里使用了OVERWRITE关键字，因此之前分区的内容将会被覆盖掉；

　　如果没有OVERWRITE关键字或者使用INTO关键字替换掉它的话，那么Hive将会以追加的方式写入数据而不会覆盖掉之前已经存在的内容；
	
　　如果表stage_employees非常大，而且用户需要对65个州都执行这些语句，那么也就意味着需要扫描stage_employees表65次！**Hive还提供了另一种INSERT语法，可以只扫描一次输入数据，然后按多种方式进行划分**。如下例子显示了如何为3个州创建表employees分区：

![Image Name](https://cdn.kesci.com/upload/image/qwhisttcz1.png?imageView2/0/w/640/h/640)

- **动态分区插入**

　　如果需要创建非常多的分区，那么用户就需要写非常多的SQL！不过，Hive提供了一个动态分区功能，其可以基于查询参数推断出需要创建的分区名称；
	
　　下面这个例子是对前面代码的修改：（country字段和state字段都是动态值）

![Image Name](https://cdn.kesci.com/upload/image/qwhmeo1m11.png?imageView2/0/w/480/h/320)

　　用户也可以混合使用动态和静态分区，如下这个例子中指定了country字段的值为静态值，而分区字段state是动态值；

![Image Name](https://cdn.kesci.com/upload/image/qwhmjkmaux.png?imageView2/0/w/480/h/320)

　　静态分区键必须在动态分区键之前；动态分区功能默认不会开启，这有助于阻止因涉及错误导致查询产生大量的分区（如时间戳，以秒作为分区，期望是以天或者小时分区）；
	
![Image Name](https://cdn.kesci.com/upload/image/qwhnl56v3.png?imageView2/0/w/640/h/640)
	

### 1.3 单个查询语句中创建表并加载数据

　　用户同样可以在一个语句中完成创建表并将查询结果载入这个表的操作：

![Image Name](https://cdn.kesci.com/upload/image/qwhnq6qp6a.png?imageView2/0/w/480/h/320)

　　使用这个功能的常见情况：从一个大的宽表中选取部分需要的数据集；但这个功能不能用于外部表，可以使用ALTER TABLE语句为外部表"引用"到一个分区，这里本身没有进行数据"装载"，而是将元数据中指定一个指向数据的路径；

### 1.4 导出数据

　　如果数据文件恰好是用户需要的格式，那么只需要简单地拷贝文件夹或者文件就可以：hadoop fs -cp source_path target_path；
	
　　否则，用户可以使用INSERT ... DIRECTORY ... ，如下面例子所示：
	
![Image Name](https://cdn.kesci.com/upload/image/qwho55odgr.png?imageView2/0/w/640/h/320)

　　用户也是可以通过如下方式指定多个输出文件夹目录：

![Image Name](https://cdn.kesci.com/upload/image/qwho8dxlvt.png?imageView2/0/w/640/h/320)


## 2. SELECT ... FROM ...语句

**知识点：**

（1）当用户选择的列是集合数据类型时，Hive会使用JSON（Java脚本对象表示法）语法应用于输出；

（2）输出：集合的字符串是加上引号的，而基本数据类型STRING的列值是不加引号的；

（3）Map数据类型的列的输出，会使用JSON格式来表示MAP，即使用一个被括在{...}内的以逗号分隔来显示键和值；

（4）STRUCT结构体类型也是以map格式输出；
　　
　　
**关于如何引用集合数据类型中的元素：**

（1）ARRAY类型：SELECT name, subordinates[0] FROM employees；

（2）MAP类型：SELECT name, deductions["State Taaxes"] FROM employees；

（3）STRUCT类型：SELECT name, address.city FROM employees；用"点"符号来表示；

### 2.1 使用正则表达式来指定列

　　从表stocks中选择symbol列和所有列名以price作为前缀的列；

In [ ]:
SELECT symbol, 'price.*' FROM stocks;

### 2.2 使用列值进行计算

　　查询转换为大写的雇员姓名、雇员对应的薪水、需要缴纳的联邦税收比例以及扣除税收后再进行取整所得的税后薪资；

In [ ]:
SELECT upper(name), salary, deductions["Federal Taxes"], round(salary * (1 - deductions["Federal Taxes"])) FROM employees;


### 2.3 算术运算符

　　Hive支持所有典型的算术运算符；
	
![Image Name](https://cdn.kesci.com/upload/image/qwht34jd52.png?imageView2/0/w/640/h/480)

　　算术运算符可以接受任意的数值类型，不过如果数据类型不同，那么两种类型中值范围较小的那个数据类型将转换为其他范围更广的数据类型，比如INT和BIGINT；INT和FLOAT；

### 2.4 使用函数（P82 - P92）

#### 2.4.1 数学函数

![Image Name](https://cdn.kesci.com/upload/image/qwhter3xfk.png?imageView2/0/w/640/h/640)

![Image Name](https://cdn.kesci.com/upload/image/qwhth8cg2l.png?imageView2/0/w/640/h/640)

![Image Name](https://cdn.kesci.com/upload/image/qwhtf9ahl1.png?imageView2/0/w/640/h/640)

![Image Name](https://cdn.kesci.com/upload/image/qwhtfhztm9.png?imageView2/0/w/640/h/640)

　　需要注意的是，函数floor、round和ceil（向上取整）输入的是DOUBLE类型的值，而返回值是BIGINT类型的；

#### 2.4.2 聚合函数

![Image Name](https://cdn.kesci.com/upload/image/qwhtpv23zr.png?imageView2/0/w/640/h/640)

![Image Name](https://cdn.kesci.com/upload/image/qwhtq4qipp.png?imageView2/0/w/640/h/640)

![Image Name](https://cdn.kesci.com/upload/image/qwhtqcwaqx.png?imageView2/0/w/640/h/640)

#### 2.4.3 表生成函数

　　表生成函数可以将单列扩展为多列或者多行；
	
　　例如：使用查询语句将employees表中每行记录中的subordinates字段内容转换为0个或者多个新的记录行，如果某行雇员记录subordinates字段为空的话，那么将不会产生新的记录；如果不为空的话，那么这个数组的每个元素都将产生一行新记录；

In [ ]:
SELECT explode(subordinates) AS sub FROM employees;

![Image Name](https://cdn.kesci.com/upload/image/qwje2yx19q.png?imageView2/0/w/640/h/640)

### 2.4.4 其他内置函数

![Image Name](https://cdn.kesci.com/upload/image/qwje6dgk20.png?imageView2/0/w/640/h/640)

![Image Name](https://cdn.kesci.com/upload/image/qwje7wrlcr.png?imageView2/0/w/760/h/760)

![Image Name](https://cdn.kesci.com/upload/image/qwjeagl5ol.png?imageView2/0/w/760/h/760)

![Image Name](https://cdn.kesci.com/upload/image/qwjebkru89.png?imageView2/0/w/760/h/760)

![Image Name](https://cdn.kesci.com/upload/image/qwjecxh7kj.png?imageView2/0/w/640/h/640)

### 2.5 LIMIT语句

　　典型的查询会返回多行数据，LIMIT子句用于限制返回的行数；

In [ ]:
SELECT
upper(name)
,salary
,deductions["Federal Taxes"]
,round(salary * (1 - deductions["Federal Taxes"]))
FROM employees
LIMIT 2;

### 2.6 列别名 -- AS

　　对第3个和第4个字段起别名，别名分别为fed_taxes和salary_minus_fed_taxes；

In [ ]:
SELECT
upper(name)
,salary
,deductions["Federal Taxes"] as fed_taxes
,round(salary * (1 - deductions["Federal Taxes"])) as salary_minus_fed_taxes
FROM employees
LIMIT 2;

### 2.7 嵌套SELECT语句

　　对于嵌套查询语句来说，使用别名是非常有用的；

In [ ]:
FROM (
    SELECT
    upper(name)
    ,salary
    ,deductions["Federal Taxes"] as fed_taxes
    ,round(salary * (1 - deductions["Federal Taxes"])) as salary_minus_fed_taxes
    FROM employees
) e
SELECT e.name, e.salary_minus_fed_taxes
WHERE e.salary_minus_fed_taxes > 70000;

### 2.8 CASE ... WHEN ... THEN 句式

　　CASE ... WHEN ... THEN 语句和if条件语句类似，用于处理单个列的查询结果；

In [ ]:
SELECT
name
,salary
,CASE
    WHEN salary < 50000.0 THEN 'low'
    WHEN salary >= 50000.0 AND salary < 70000.0 THEN 'middle'
    WHEN salary >= 70000.0 AND salary < 100000.0 THEN 'high'
    ELSE 'very high'
END AS bracket
FROM employees;

### 2.9 什么情况下Hive可以避免进行MapReduce

![Image Name](https://cdn.kesci.com/upload/image/qwjfdbs5nn.png?imageView2/0/w/640/h/640)


## 3. WHERE语句

### 3.1 谓词操作符

![Image Name](https://cdn.kesci.com/upload/image/qwjg0nri91.png?imageView2/0/w/960/h/640)

### 3.2 关于浮点数比较（FLOAT和DOUBLE）

　　BackGround：

![Image Name](https://cdn.kesci.com/upload/image/qwjh9uwir7.png?imageView2/0/w/640/h/640)

　　Hive会隐式地将FLOAT类型的字段转换为DOUBLE类型后再通过WHERE与小数（比如0.2）进行比较，这样可以吗？**事实上，这是行不通的**！因为，0.2对于FLOAT类型是0.2000001，而对于DOUBLE类型是0.200000000001；当表中的FLOAT值通过Hive转换为DOUBLE值时，其产生的DOUBLE值是0.200000100000，这个值实际上比0.200000000001大，这就是为什么上述输出结果错误的原因了；
　　
	　　
　　那么怎么规避这个问题呢？Hive中有两种方法；

　（1）方法一：从TEXTFILE文本文件读取字符串"0.2"，然后将其转换为一个真实的数字，可以在表模式中定义对应的字段类型为DOUBLE 而不是FLOAT；

　（2）方法二：显式地指出0.2为FLOAT类型，这里就必须使用**cast操作符**；


In [ ]:
SELECT
name
,salary
,deductions['Federal Taxes']
FROM employees
WHERE deductions['Federal Taxes'] > cast(0.2 AS FLOAT);

### 3.3 LIKE和RLIKE

　　LIKE的话，使用方法跟SQL是一样的，这里就不写案例了，只是简单写几个匹配：**以字符串Ave结尾（%Ave.）；以O开头（O%）；包含Chicago（%Chicago%）**；

　　**RLIKE**子句是Hive中这个功能的一个扩展，其可以通过Java的正则表达式这个更强大的语言来指定匹配条件（书中没有过多涉及正则表达式），书中给出了以下这个例子：

　　eg：**从employees表中查找所有住址的街道名称中含有单词Chicago或Ontario的雇员名称和街道信息；**

In [ ]:
SELECT
name
,address.street
FROM employees 
WHERE address.street RLIKE '.*(Chicago|Ontario).*';
# 上述这个RLIKE语句如果换成LIKE的话：
# address.street LIKE '%Chicago%' OR address.street LIKE '%Ontario%';

## 4. GROUP BY 语句

　　GROUP BY 语句通常会和聚合函数一起使用，按照一个或者多个列对结果进行分组，然后对每个组执行聚合操作；跟SQL也是一样用法，这里就省略了；
	
- **HAVING 语句**

　　HAVING语句允许用户通过一个简单的语法完成原本需要通过子查询才能对GROUP BY语句产生的分组进行条件过滤的任务；（略）

## 5. JOIN语句

　　HIVE支持通常的SQL JOIN语句，但是只支持等值连接；

### 5.1 INNER JOIN

　　内连接（INNER JOIN）中，只有进行连接的两个表中都存在与连接标准相匹配的数据才会保留下来；
	
　　SELECT ... FROM  ... JOIN ... ON ... WHERE ... AND ...

　　标准SQL是支持对连接关键词进行非等值连接的；比如下面这个：
	
![Image Name](https://cdn.kesci.com/upload/image/qwou95momu.png?imageView2/0/w/640/h/320)

　　然后，**这在Hive中是非法的！**主要原因是通过MapReduce很难实现这种类型的连接。同时Hive目前还不支持在ON子句中的谓词间使用OR；

### 5.2 JOIN优化

　　当三个表进行内连接时，这里假定其中一个表的某个字段作为连接另外两个表的**JOIN连接键**；在这种情况下，Hive通过一个**优化**可以在同一个MapReduce job中连接3张表；
	
　　【提示】当对3个或者更多的表进行JOIN连接时，如果每个ON子句都使用相同的连接键的话，那么只会产生一个MapReduce job；
	
　　Hive同时假定查询中最后一个表时最大的那个表。因此，用户需要保证连续查询中的表的大小从左到右是依次增加的；不过，Hive提供了一个**"标记"机制**来显式地告之查询优化器哪张表是大表；

![Image Name](https://cdn.kesci.com/upload/image/qwov9yno78.png?imageView2/0/w/640/h/320)

　　还有另外一个类似的非常重要的优化叫做map-side JOIN，可以参考5.9节；

### 5.3 LEFT OUTER JOIN

　　左外连接通过关键字LEFT OUTER进行标识；（跟SQL一样，略）

### 5.4 OUTER JOIN

　　WHERE过滤条件对于外连接（OUTER JOIN）会忽略掉分区过滤条件，不过对于内连接（INNER JOIN）使用这样的过滤词确实会起作用的！
	
　　幸运的是，有一个适用于所有种类连接的解决方案，那就是使用嵌套SELECT语句；
	
![Image Name](https://cdn.kesci.com/upload/image/qwowx1pxrs.png?imageView2/0/w/640/h/320)

### 5.5 RIGHT OUTER JOIN

　　**右外连接（RIGHT  OUTER JOIN）**会返回右边表所有符合WHERE语句的记录；左表中匹配不上的字段值用NULL代替；（代码略）

### 5.6 FULL OUTER JOIN

　　**完全外连接（FULL OUTER JOIN）**将会返回所有表中符合WHERE语句条件的所有记录；如果任一表的指定字段没有符合条件的值得话，那么就使用NULL值；（代码略）

### 5.7 LEFT SEMI-JOIN

　　**左半开连接（LEFT SEMI-JOIN）**会返回左边表的记录，前提是其记录对于右边表满足ON语句中的判定条件；
	
![Image Name](https://cdn.kesci.com/upload/image/qwoyicglha.png?imageView2/0/w/640/h/320)

　　不过，可以使用如下的LEFT SEMO JOIN 语法达到同样的目的：
	
![Image Name](https://cdn.kesci.com/upload/image/qwoyily78n.png?imageView2/0/w/640/h/320)

　　请注意，SELECT和WHERE语句中不能引用到右边表中的字段；
	
　　【注意】Hive不支持右半开连接（RIGHT SEMI-JOIN）
	
　　SEMI-JOIN比通常的INNER JOIN要更高效，原因如下：对于左表中一条指定的记录,在右边表中一旦找到匹配的记录，Hive就会立即停止扫描。从这点来看，左边表中选择的列是可以预测的。


### 5.8 笛卡尔积JOIN

　　**笛卡尔积**是一种连接，表示左边表的行数乘以右边表的行数等于笛卡尔结果集的大小。如果使用了错误的连接（JOIN）语法可能会导致产生一个执行时间长、运行缓慢的笛卡尔积查询。比如通过使用where子句来连接，一般在很多数据库中会被优化成内连接（INNER JOIN），但是在Hive中没有此优化；
	
　　笛卡尔积的一个应用举例：假设有一个表表示用户偏好，另有一个表白哦是新闻文章，同时有一个算法会推测出用户可能会喜欢读哪些文章。这个时候就需要使用笛卡尔积生成所有用户和网页的对应关系的集合；

### 5.9 map-side JOIN

　　如果所有表中只有一张表是小表，那么可以在最大的表通过mapper的时候将小表完全放到内存中。Hive可以在map端执行连接过程（称为map-side JOIN），这是因为Hive可以和内存中的小表进行逐一匹配，从而省略掉常规连接操作所需要的reduce过程；

　　关于map-side JOIN如何使用：

![Image Name](https://cdn.kesci.com/upload/image/qwvrboyypt.png?imageView2/0/w/640/h/640)

## 6. ORDER BY和SORT BY

　　Hive中**ORDER BY**语句和其他的SQL方言中的定义是一样的，其会对查询结果集执行一个**全局排序**。这也就是说会有一个所有的数据都通过一个reducer进行处理的过程。对于**大**数据集，这个过程可能会**消耗太过漫长的时间**来执行；

　　Hive增加了一个可供选择的方式，也就是**SORT BY**，其只会在每个reducer中对数据进行排序，也就是执行一个**局部排序**的过程。这可以保证每个reducer的输出数据都是有序的（**但并非全局有序**）。这样可以提高后面进行的**全局排序**的效率；

　　当reducer的个数等于1时，order by和sort by的输出结果是一样的；但是当reducer的个数大于1时，输出结果的排序就不一样了；

## 7. 含有SORT BY的DISTRIBUTE BY

　　**DISTRIBUTE BY控制map的输出在reducer中是如何划分的**。MapReducer job中传输的所有数据都是按照键-值对的方式进行组织的，因此Hive在将用户的查询语句转换成MapReduce job时，其必须在内部使用这个功能；

　　**举个栗子**：假设我们希望具有相同股票交易码的数据在一起处理。那么我们可以使用**DISTRIBUTE BY**来保证具有相同股票交易码的记录会分发到**同一个reducer中**进行处理，然后使用**SORT BY**来按照我们的期望对数据进行排序，代码如下；

In [ ]:
SELECT s.ymd, s.symbol, s.price_close
FROM stocks s
DISTRIBUTE BY s.symbol
SORT BY s.symbol ASC, s.ymd ASC; 

　　DISTRIBUTE BY和GROUP BY在其控制着reducer是如何接受一行行数据进行处理这方面是类似的，而SORT BY则控制着reducer内的数据是如何进行排序的；需要注意的是，Hive要求DISTRIBUTE BY语句要写在SORT BY语句之前；

## 8. CLUSTER BY

　　在前面的例子中，s.symbol列被用在了**DISTRIBUTE B**Y语句中，而s.symbol列和s.ymd位于**SORT BY**语句中。如果**这2个语句中涉及到的列完全相同**，而且采用的是**升序排序**方式（也就是默认的排序方式），那么在这种情况下，**CLUSTER BY**就等价于前面的2个语句，相当于前面2个句子的一个简写方式；

In [ ]:
SELECT s.ymd, s.symbol, s.price_close
FROM stocks s
CLUSTER BY s.symbol;

　　使用**DISTRIBUTE BY ... SORT BY语**句或其简化版的**CLUSTER BY语句**会剥夺**SORT BY的并行性**，然后这样可以实现输出文件的数据是**全局排序**的。

## 9. 类型转换

　　用户可以使用**cast()函数**对指定的值进行显式的类型转换；类型转换函数的语法是**cast(value AS TYPE)**，需要注意的是，将浮点数转换成整数最好使用**round()函数**或者**floor()函数**，而不是cast()函数；
	
- **类型转换BINARY值**

　　Hive v0.8.0版本中新引入的BINARY类型只支持将BINARY类型转换为STRING类型。不过，如果用户知道其值是数值的话，那么可以通过嵌套cast()的方式对其进行类型转换，如下例子：

In [ ]:
SELECT (2.0*cast(cast(b as string) as double))
FROM src;
# 用户同样可以将STRING类型转换为BINARY类型;

## 10. 抽样查询

　　对于非常大的数据集，有时用户需要使用的是一个具有代表性的查询结果而不是全部结果，Hive可以通过对表进行分桶抽样来满足这个需求；
	
　　我们可以使用rand()函数进行抽样，这个函数会返回一个随机值。前两个查询都返回了两个不相等的值，而第3个查询语句无返回结果；
	
![Image Name](https://cdn.kesci.com/upload/image/qwvuw4lns7.png?imageView2/0/w/640/h/320)

　　如果我们是按照指定的列而非rand()函数进行分桶的话，那么同一语句多次执行的返回值是相同的：

![Image Name](https://cdn.kesci.com/upload/image/qwvv15cny0.png?imageView2/0/w/640/h/320)

![Image Name](https://cdn.kesci.com/upload/image/qwvv1enz9f.png?imageView2/0/w/640/h/320)

　　分桶语句中的分母表示的是数据将会被散列的桶的个数，而分子表示将会选择的桶的个数：

![Image Name](https://cdn.kesci.com/upload/image/qwvv53bpft.png?imageView2/0/w/640/h/640)



### 10.1 数据块抽样

　　Hive提供了另外一种按照**抽样百分比**进行抽样的方式，这种是基于**行数**的，按照输入路径下的数据块百分比进行的抽样：

In [ ]:
SELECT * FROM numbersflat TABLESAMPLE(0.1 PERCENT) s;

　【提示】这种抽样方式不一定适用于所有的文件格式。另外，这种抽样的最小抽样单元是一个HDFS数据块。因此，如果表的数据大小小于普通的块大小128MB的话，那么将会返回所有行；

### 10.2 分桶表的输入裁剪

　　如果TABLESAMPLE语句中指定的列和CLUSTERED BY语句中指定的列相同，那么TABLESAMPLE查询就只会扫描涉及到的表的哈斯分区下的数据；（没看懂，先留个印象，等用到的时候在过来细细看下）

## 11. UNION ALL

　　**UNION ALL**可以将**2个或多个表**进行合并。每一个union子查询都必需具有**相同的列**，而且对应的**每个字段的字段类型必须是一致的**。例如，如果第2个字段是FLOAT类型的，那么所有其他子查询的第2个字段必须都是FLOAT类型的。

In [ ]:
SELECT log.ymd, log.level, log.message
FROM (
        SELECT l1.ymd, l1.level, l1.message, 'Log1' AS source
        FROM log1 l1
    UNION ALL
        SELECT l2.ymd, l2.level, l2.message, 'Log2' AS source
        FROM log1 l2
) log
SORT BY log.ymd ASC;

　　UNION也可用于**同一个源表的数据合并**。从逻辑上讲，可以使用一个SELECT和WHERE语句来获得相同的结果。这个技术便于将一个长的复杂的WHERE语句分割成 2个或者多个UNION子查询。不过，除非源表建立了索引，否则，这个查询将会对同一份源数据进行多次拷贝分发。例如：

In [ ]:
FROM(
    FROM src SELECT src.key, src.value WHERE src.key < 100
    UNION ALL
    FROM src SELECT src.* WHERE src.key > 110
) unioninput
INSERT OVERWRITE DIRECTORY '/tmp/union.out' SELECT unioninput.*;